# TSA Chapter 5: Stylized Facts

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch5/TSA_ch5_stylized/TSA_ch5_stylized.ipynb)

Stylized facts of financial returns: ACF, fat tails, and QQ-plot.

In [ ]:
!pip install numpy pandas matplotlib scipy yfinance arch statsmodels -q

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy import stats
# Chart style
BLUE = '#1A3A6E'; RED = '#DC3545'; GREEN = '#2E7D32'
ORANGE = '#E67E22'; GRAY = '#666666'; PURPLE = '#8E44AD'

plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none',
    'savefig.facecolor': 'none', 'savefig.transparent': True,
    'axes.grid': False, 'font.size': 10, 'axes.labelsize': 11,
    'axes.titlesize': 12, 'legend.fontsize': 9, 'xtick.labelsize': 9,
    'ytick.labelsize': 9, 'axes.spines.top': False, 'axes.spines.right': False,
    'lines.linewidth': 1.0, 'axes.linewidth': 0.6,
    'legend.facecolor': 'none', 'legend.framealpha': 0, 'legend.edgecolor': 'none',
    'figure.dpi': 150,
})

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}')

def legend_bottom(ax, ncol=2, y=-0.18):
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, y), ncol=ncol, frameon=False)
from statsmodels.graphics.tsaplots import plot_acf

In [ ]:
import yfinance as yf

# Download S&P 500 data
sp = yf.download('^GSPC', start='2000-01-01', end='2025-12-31', progress=False)
if isinstance(sp.columns, pd.MultiIndex):
    sp.columns = sp.columns.get_level_values(0)
sp500_returns = (sp['Close'].pct_change() * 100).dropna()
sp500_returns = pd.Series(sp500_returns.values, index=sp500_returns.index, name='returns')
print(f"S&P 500: {len(sp500_returns)} observations")

In [ ]:
# Stylized facts of S&P 500 returns
fig, axes = plt.subplots(2, 2, figsize=(12, 9))

plot_acf(sp500_returns.values, lags=30, ax=axes[0, 0], color=BLUE, vlines_kwargs={'color': BLUE}, alpha=0.05)
axes[0, 0].set_title('ACF of S&P 500 Returns', fontweight='bold'); axes[0, 0].set_xlabel('Lag')

plot_acf(sp500_returns.values**2, lags=30, ax=axes[0, 1], color=RED, vlines_kwargs={'color': RED}, alpha=0.05)
axes[0, 1].set_title('ACF of Squared Returns', fontweight='bold'); axes[0, 1].set_xlabel('Lag')

axes[1, 0].hist(sp500_returns.values, bins=100, density=True, color=BLUE, alpha=0.7, edgecolor='white', label='S&P 500')
x = np.linspace(sp500_returns.min(), sp500_returns.max(), 200)
axes[1, 0].plot(x, stats.norm.pdf(x, sp500_returns.mean(), sp500_returns.std()), color=RED, lw=2, label='Normal')
axes[1, 0].set_title('Distribution: Fat Tails', fontweight='bold')
axes[1, 0].set_xlabel('Return (%)'); axes[1, 0].set_ylabel('Density'); axes[1, 0].set_xlim(-10, 10)
kurt = float(stats.kurtosis(sp500_returns.values)); skew = float(stats.skew(sp500_returns.values))
axes[1, 0].text(0.95, 0.95, f'Kurtosis: {kurt:.2f}\nSkewness: {skew:.2f}',
                transform=axes[1, 0].transAxes, fontsize=9, va='top', ha='right',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
axes[1, 0].legend(loc='upper left', frameon=False)

ret_flat = sp500_returns.values.flatten()
stats.probplot(ret_flat, dist="norm", plot=axes[1, 1])
axes[1, 1].get_lines()[0].set_color(BLUE); axes[1, 1].get_lines()[0].set_markersize(2)
axes[1, 1].get_lines()[1].set_color(RED)
axes[1, 1].set_title('QQ-Plot vs Normal', fontweight='bold')

fig.tight_layout(); save_chart(fig, 'garch_stylized_facts'); plt.show()